here we can get the proposals and votes at the same time

we have to paginate, otherwise the request is too big and it breaks

In [245]:
import requests

headers = {
    'authority': 'api.tally.xyz',
    'accept': '*/*',
    'accept-language': 'es-ES,es;q=0.5',
    'api-key': 'a0a4cd00bb6953720c9c201c010cdd36a563e65c97e926a36a8acdfcd1d1eeb7',
    'cache-control': 'no-cache',
    'content-type': 'application/json',
    'origin': 'https://www.tally.xyz',
    'pragma': 'no-cache',
    'referer': 'https://www.tally.xyz/',
    'sec-ch-ua': '"Not.A/Brand";v="8", "Chromium";v="114", "Brave";v="114"',
    'sec-ch-ua-mobile': '?1',
    'sec-ch-ua-platform': '"Android"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-site',
    'sec-gpc': '1',
    'user-agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Mobile Safari/537.36',
}

query = '''
    query GovernanceProposals($sort: ProposalSort, $chainId: ChainID!, $pagination: Pagination, $governanceIds: [AccountID!], $proposerIds: [AccountID!], $voters: [Address!], $votersPagination: Pagination, $includeVotes: Boolean!) {
    proposals(
        sort: $sort
        chainId: $chainId
        pagination: $pagination
        governanceIds: $governanceIds
        proposerIds: $proposerIds
      ) {
        id
        # description
        # statusChanges {
            # type
        # }
        block {
            timestamp
        }
        # voteStats {
        #   votes
        #   weight
        #   support
        #   percent
        # }
        votes(voters: $voters, pagination: $votersPagination) @include(if: $includeVotes) {
            id
            voter {
                # name
                # picture
                address
                # identities {
                    # twitter
                # }
            }
            block {
                timestamp
            }
            weight
            support
        }
        governance {
          id
        #   quorum
        #   name
        #   timelockId
        #   organization {
                # visual {
                #   icon
            # }
        #   }
        #   tokens {
        #         decimals
        #   }
        }
        # tallyProposal {
        #   id
        #   createdAt
        #   status
        # }
      }
    }
'''

json_data = {
    'query': query,
    'variables': {
        'pagination': {
            'limit': 11,
            'offset': 0,
        },
        'sort': {
            'field': 'START_BLOCK',
            'order': 'DESC',
        },
        'chainId': 'eip155:10',
        'governanceIds': [
            'eip155:10:0xcDF27F107725988f2261Ce2256bDfCdE8B382B10',
        ],
        'votersPagination': {
            # 'limit': 1,
            # 'offset': 0,
        },
        'includeVotes': True,
    },
}

response = requests.post('https://api.tally.xyz/query', headers=headers, json=json_data)

# Note: json_data will not be serialized by requests
# exactly as it was in the original request.
#data = '{"query":"\\n    query GovernanceProposals($sort: ProposalSort, $chainId: ChainID!, $pagination: Pagination, $governanceIds: [AccountID!], $proposerIds: [AccountID!], $voters: [Address!], $votersPagination: Pagination, $includeVotes: Boolean!) {\\n  proposals(\\n    sort: $sort\\n    chainId: $chainId\\n    pagination: $pagination\\n    governanceIds: $governanceIds\\n    proposerIds: $proposerIds\\n  ) {\\n    id\\n    description\\n    statusChanges {\\n      type\\n    }\\n    block {\\n      timestamp\\n    }\\n    voteStats {\\n      votes\\n      weight\\n      support\\n      percent\\n    }\\n    votes(voters: $voters, pagination: $votersPagination) @include(if: $includeVotes) {\\n      support\\n    }\\n    governance {\\n      id\\n      quorum\\n      name\\n      timelockId\\n      organization {\\n        visual {\\n          icon\\n        }\\n      }\\n      tokens {\\n        decimals\\n      }\\n    }\\n    tallyProposal {\\n      id\\n      createdAt\\n      status\\n    }\\n  }\\n}\\n    ","variables":{"pagination":{"limit":10,"offset":0},"sort":{"field":"START_BLOCK","order":"DESC"},"chainId":"eip155:10","governanceIds":["eip155:10:0xcDF27F107725988f2261Ce2256bDfCdE8B382B10"],"votersPagination":{"limit":1,"offset":0},"includeVotes":false}}'
#response = requests.post('https://api.tally.xyz/query', headers=headers, data=data)

In [246]:
response.json()

{'data': {'proposals': [{'id': '80982553847843251343725022866904947381762263529096361834044805234222094077710',
    'block': {'timestamp': '2023-05-18T18:16:06Z'},
    'votes': [{'id': 'eip155:10:0xcDF27F107725988f2261Ce2256bDfCdE8B382B10/80982553847843251343725022866904947381762263529096361834044805234222094077710|eip155:10:0x4ed0f6Fe7156eFEDD299812b8D27F60a6C70fF6D',
      'voter': {'address': '0x4ed0f6Fe7156eFEDD299812b8D27F60a6C70fF6D'},
      'block': {'timestamp': '2023-05-18T18:28:25Z'},
      'weight': '210000000000000000000',
      'support': 'AGAINST'}],
    'governance': {'id': 'eip155:10:0xcDF27F107725988f2261Ce2256bDfCdE8B382B10'}},
   {'id': '29831001453379581627736734765818959389842109811221412662144194715522205098015',
    'block': {'timestamp': '2023-05-18T18:16:06Z'},
    'votes': [],
    'governance': {'id': 'eip155:10:0xcDF27F107725988f2261Ce2256bDfCdE8B382B10'}},
   {'id': '94365805422398770067924881378455503928423439630602149628781926844759467250082',
    'block':

In [247]:
# load response to df
import pandas as pd
df = pd.DataFrame(response.json()['data']['proposals'])
df.shape

(11, 4)

keep only proposals for our time period

In [248]:
# map block.timestamp to proposalCreatedAt column
df['proposalCreatedAt'] = df['block'].apply(lambda x: x['timestamp'])
# remove block column
df.drop(columns=['block'], inplace=True)
df.head()

,id,votes,governance,proposalCreatedAt
0,8098255384784325134372502286690494738176226352...,[{'id': 'eip155:10:0xcDF27F107725988f2261Ce225...,{'id': 'eip155:10:0xcDF27F107725988f2261Ce2256...,2023-05-18T18:16:06Z
1,2983100145337958162773673476581895938984210981...,[],{'id': 'eip155:10:0xcDF27F107725988f2261Ce2256...,2023-05-18T18:16:06Z
2,9436580542239877006792488137845550392842343963...,[{'id': 'eip155:10:0xcDF27F107725988f2261Ce225...,{'id': 'eip155:10:0xcDF27F107725988f2261Ce2256...,2023-04-27T19:30:36Z
3,1036953249134245978023891813127229930376010326...,[{'id': 'eip155:10:0xcDF27F107725988f2261Ce225...,{'id': 'eip155:10:0xcDF27F107725988f2261Ce2256...,2023-04-27T19:30:36Z
4,3297070190487044661440837301194291768042237675...,[{'id': 'eip155:10:0xcDF27F107725988f2261Ce225...,{'id': 'eip155:10:0xcDF27F107725988f2261Ce2256...,2023-04-27T19:30:36Z


In [249]:
# make createdAt column datetime parse format "2023-05-18T18:16:06Z"
df['proposalCreatedAt'] = pd.to_datetime(df['proposalCreatedAt'], format="%Y-%m-%dT%H:%M:%SZ")
df.head()

,id,votes,governance,proposalCreatedAt
0,8098255384784325134372502286690494738176226352...,[{'id': 'eip155:10:0xcDF27F107725988f2261Ce225...,{'id': 'eip155:10:0xcDF27F107725988f2261Ce2256...,2023-05-18 18:16:06
1,2983100145337958162773673476581895938984210981...,[],{'id': 'eip155:10:0xcDF27F107725988f2261Ce2256...,2023-05-18 18:16:06
2,9436580542239877006792488137845550392842343963...,[{'id': 'eip155:10:0xcDF27F107725988f2261Ce225...,{'id': 'eip155:10:0xcDF27F107725988f2261Ce2256...,2023-04-27 19:30:36
3,1036953249134245978023891813127229930376010326...,[{'id': 'eip155:10:0xcDF27F107725988f2261Ce225...,{'id': 'eip155:10:0xcDF27F107725988f2261Ce2256...,2023-04-27 19:30:36
4,3297070190487044661440837301194291768042237675...,[{'id': 'eip155:10:0xcDF27F107725988f2261Ce225...,{'id': 'eip155:10:0xcDF27F107725988f2261Ce2256...,2023-04-27 19:30:36


In [250]:
YEAR_AGO_EPOCH = 1656021600  # 2022-06-24 00:00:00
NOW_EPOCH =      1687557600  # 2023-06-24 00:00:00
# make these into datetimes
YEAR_AGO_EPOCH = pd.to_datetime(YEAR_AGO_EPOCH, unit='s')
NOW_EPOCH = pd.to_datetime(NOW_EPOCH, unit='s')

# filter df by createdAt column
df = df[(df['proposalCreatedAt'] > YEAR_AGO_EPOCH) & (df['proposalCreatedAt'] < NOW_EPOCH)]
# drop proposalCreatedAt column
df.drop(columns=['proposalCreatedAt'], inplace=True)
df.shape

(11, 4)

In [251]:
df.iloc[0]

id                   8098255384784325134372502286690494738176226352...
votes                [{'id': 'eip155:10:0xcDF27F107725988f2261Ce225...
governance           {'id': 'eip155:10:0xcDF27F107725988f2261Ce2256...
proposalCreatedAt                                  2023-05-18 18:16:06
Name: 0, dtype: object

In [252]:
df.head()

,id,votes,governance,proposalCreatedAt
0,8098255384784325134372502286690494738176226352...,[{'id': 'eip155:10:0xcDF27F107725988f2261Ce225...,{'id': 'eip155:10:0xcDF27F107725988f2261Ce2256...,2023-05-18 18:16:06
1,2983100145337958162773673476581895938984210981...,[],{'id': 'eip155:10:0xcDF27F107725988f2261Ce2256...,2023-05-18 18:16:06
2,9436580542239877006792488137845550392842343963...,[{'id': 'eip155:10:0xcDF27F107725988f2261Ce225...,{'id': 'eip155:10:0xcDF27F107725988f2261Ce2256...,2023-04-27 19:30:36
3,1036953249134245978023891813127229930376010326...,[{'id': 'eip155:10:0xcDF27F107725988f2261Ce225...,{'id': 'eip155:10:0xcDF27F107725988f2261Ce2256...,2023-04-27 19:30:36
4,3297070190487044661440837301194291768042237675...,[{'id': 'eip155:10:0xcDF27F107725988f2261Ce225...,{'id': 'eip155:10:0xcDF27F107725988f2261Ce2256...,2023-04-27 19:30:36


In [253]:
# explode votes
df = df.explode('votes')
df.shape

(230759, 4)

In [254]:
df.head()

,id,votes,governance,proposalCreatedAt
0,8098255384784325134372502286690494738176226352...,{'id': 'eip155:10:0xcDF27F107725988f2261Ce2256...,{'id': 'eip155:10:0xcDF27F107725988f2261Ce2256...,2023-05-18 18:16:06
1,2983100145337958162773673476581895938984210981...,NaN,{'id': 'eip155:10:0xcDF27F107725988f2261Ce2256...,2023-05-18 18:16:06
2,9436580542239877006792488137845550392842343963...,{'id': 'eip155:10:0xcDF27F107725988f2261Ce2256...,{'id': 'eip155:10:0xcDF27F107725988f2261Ce2256...,2023-04-27 19:30:36
2,9436580542239877006792488137845550392842343963...,{'id': 'eip155:10:0xcDF27F107725988f2261Ce2256...,{'id': 'eip155:10:0xcDF27F107725988f2261Ce2256...,2023-04-27 19:30:36
2,9436580542239877006792488137845550392842343963...,{'id': 'eip155:10:0xcDF27F107725988f2261Ce2256...,{'id': 'eip155:10:0xcDF27F107725988f2261Ce2256...,2023-04-27 19:30:36


In [255]:
# rename id to proposalId
df.rename(columns={'id': 'proposalId'}, inplace=True)
# make organizationId coluimn from governance.id
df['organizationId'] = df['governance'].apply(lambda x: x['id'])
# drop governance
df.drop('governance', axis=1, inplace=True)

# expand votes
df = pd.concat([df.drop(['votes'], axis=1), df['votes'].apply(pd.Series)], axis=1)
df.head()

,proposalId,proposalCreatedAt,organizationId,id,voter,block,weight,support,0
0,8098255384784325134372502286690494738176226352...,2023-05-18 18:16:06,eip155:10:0xcDF27F107725988f2261Ce2256bDfCdE8B...,eip155:10:0xcDF27F107725988f2261Ce2256bDfCdE8B...,{'address': '0x4ed0f6Fe7156eFEDD299812b8D27F60...,{'timestamp': '2023-05-18T18:28:25Z'},210000000000000000000,AGAINST,NaN
1,2983100145337958162773673476581895938984210981...,2023-05-18 18:16:06,eip155:10:0xcDF27F107725988f2261Ce2256bDfCdE8B...,NaN,NaN,NaN,NaN,NaN,NaN
2,9436580542239877006792488137845550392842343963...,2023-04-27 19:30:36,eip155:10:0xcDF27F107725988f2261Ce2256bDfCdE8B...,eip155:10:0xcDF27F107725988f2261Ce2256bDfCdE8B...,{'address': '0xfFFd52ec814476cD003a088421b92E8...,{'timestamp': '2023-04-29T09:24:50Z'},46220744581675089445,FOR,NaN
2,9436580542239877006792488137845550392842343963...,2023-04-27 19:30:36,eip155:10:0xcDF27F107725988f2261Ce2256bDfCdE8B...,eip155:10:0xcDF27F107725988f2261Ce2256bDfCdE8B...,{'address': '0xfFfb43b6b6048AdC32DeD4FcEDc429B...,{'timestamp': '2023-04-28T06:49:23Z'},15655501983199607598,FOR,NaN
2,9436580542239877006792488137845550392842343963...,2023-04-27 19:30:36,eip155:10:0xcDF27F107725988f2261Ce2256bDfCdE8B...,eip155:10:0xcDF27F107725988f2261Ce2256bDfCdE8B...,{'address': '0xFfF9E8387F6Ea2073D1C38b9571FE4a...,{'timestamp': '2023-04-28T06:49:08Z'},18791400874120817415,FOR,NaN


In [256]:
# add voter
df['voter'] = df['voter'].apply(lambda x: x['address'] if not pd.isnull(x) else None)
# # add createdAt from votes.block.timestamp
df['createdAt'] = df['block'].apply(lambda x: x['timestamp'] if not pd.isnull(x) else None)
# drop block
df.drop('block', axis=1, inplace=True)
# drop column 0
df.drop(0, axis=1, inplace=True)
df.head()

,proposalId,proposalCreatedAt,organizationId,id,voter,weight,support,createdAt
0,8098255384784325134372502286690494738176226352...,2023-05-18 18:16:06,eip155:10:0xcDF27F107725988f2261Ce2256bDfCdE8B...,eip155:10:0xcDF27F107725988f2261Ce2256bDfCdE8B...,0x4ed0f6Fe7156eFEDD299812b8D27F60a6C70fF6D,210000000000000000000,AGAINST,2023-05-18T18:28:25Z
1,2983100145337958162773673476581895938984210981...,2023-05-18 18:16:06,eip155:10:0xcDF27F107725988f2261Ce2256bDfCdE8B...,NaN,None,NaN,NaN,None
2,9436580542239877006792488137845550392842343963...,2023-04-27 19:30:36,eip155:10:0xcDF27F107725988f2261Ce2256bDfCdE8B...,eip155:10:0xcDF27F107725988f2261Ce2256bDfCdE8B...,0xfFFd52ec814476cD003a088421b92E8ddDC7A2C2,46220744581675089445,FOR,2023-04-29T09:24:50Z
2,9436580542239877006792488137845550392842343963...,2023-04-27 19:30:36,eip155:10:0xcDF27F107725988f2261Ce2256bDfCdE8B...,eip155:10:0xcDF27F107725988f2261Ce2256bDfCdE8B...,0xfFfb43b6b6048AdC32DeD4FcEDc429B17e7ea72d,15655501983199607598,FOR,2023-04-28T06:49:23Z
2,9436580542239877006792488137845550392842343963...,2023-04-27 19:30:36,eip155:10:0xcDF27F107725988f2261Ce2256bDfCdE8B...,eip155:10:0xcDF27F107725988f2261Ce2256bDfCdE8B...,0xFfF9E8387F6Ea2073D1C38b9571FE4aA55fd3ad8,18791400874120817415,FOR,2023-04-28T06:49:08Z


In [257]:
df.shape

(230759, 8)